In [2]:
from bs4 import BeautifulSoup
import datetime
import pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from google.colab import files
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import plotly
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier

1.) CSV containing games from 2015+ (1 for all stats)
2.) CSV with all player stats by season
3.) CSV with all team stats by season

# Creating CSV with all games from 2015+

Scraping team data from 2015 and later

In [2]:
def date_to_str(date):
  '''
  Args:
        date (datetime): datetime object for the day of the season

  Returns:
        str: string representation of the given date

  '''
  month = str(date.month)
  day = str(date.day)
  if date.day <= 9:
    day = str(0) + day
  if date.month <= 9:
    month = str(0) + month
  return str(date.year) + '-' + month + '-' + day

Scraping process takes ~20 minutes; CSV stored for convenience

In [3]:
def collect_team_data():
  '''
    Scrapes FanGraphs data from each day between April 1, 2015 and today's date

    Returns:
        hit (pd.DataFrame) contains hitting stats with each record representing one game for a team
        pit (pd.DataFrame) contains pitching stats with each record representing one game for a team
'''
  # beginning of sample is 2015
  first_date = datetime.datetime(year = 2015, month = 4, day = 1)
  # When date reaches last date, date resets to first_date (plus one year)
  last_date = datetime.datetime(year = 2015, month = 10, day = 3)
  date = datetime.datetime(year = 2015, month = 4, day = 1)
  # collects team hitting stats for each day
  hit = pd.DataFrame()
  # collects team pitching stats for each day
  pit = pd.DataFrame()
  # sustainable way of changing year without change in code
  while (date < datetime.datetime.now()):
      date_str = date_to_str(date)
      # scrape hitting data
      hit_df = pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=8&season={date.year}&month=1000&season1={date.year}&ind=0&team=0%2Cts&rost=0&age=0&filter=&players=0&startdate={date_str}&enddate={date_str}')
      # getting rid of the final row with non-numeric data
      hit_df = hit_df[16][:-1]
      hit_df[('temp', 'Date')] = date_str
      hit_df.columns = hit_df.columns.droplevel(0)
      if len(hit_df['#']) > 1:
        hit = hit.append(hit_df)
      # scrape pitching data
      pit_df = pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=8&season={date.year}&month=1000&season1={date.year}&ind=0&team=0%2Cts&rost=0&age=0&filter=&players=0&startdate={date_str}&enddate={date_str}')
      # getting rid of the final row with non-numeric data
      pit_df = pit_df[16][:-1]
      pit_df[('temp', 'Date')] = date_str
      pit_df.columns = pit_df.columns.droplevel(0)
      if len(pit_df['#']) > 1:
        pit = pit.append(pit_df)
      if (date < last_date):
        date += datetime.timedelta(days = 1)
      else:
        print(date.year)
        last_date = datetime.datetime(year = last_date.year + 1, month = last_date.month, day = last_date.day)
        first_date = datetime.datetime(year = first_date.year + 1, month = first_date.month, day = first_date.day)
        date = first_date
  return hit, pit

In [ ]:
# CODE USED FOR INITIAL SCRAPING

# hit, pit = collect_team_data()

In [4]:
def collect_new_team_data(df):
  '''
    Scrapes FanGraphs data from each day the most recent record scraped and today's date

    Returns:
        hit (pd.DataFrame) contains hitting stats with each record representing one game for a team
        pit (pd.DataFrame) contains pitching stats with each record representing one game for a team
'''
  recent_record = datetime.datetime.strptime(df['Date'].max(), '%Y-%m-%d')
  # beginning of sample is the most recent day data was collected
  date = recent_record + datetime.timedelta(days = 1)
  # collects team hitting stats for each day
  hit = pd.DataFrame()
  # collects team pitching stats for each day
  pit = pd.DataFrame()
  # sustainable way of changing year without change in code
  while (date < datetime.datetime.now()):
      date_str = date_to_str(date)
      # scrape hitting data
      hit_df = pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=8&season={date.year}&month=1000&season1={date.year}&ind=0&team=0%2Cts&rost=0&age=0&filter=&players=0&startdate={date_str}&enddate={date_str}')
      # getting rid of the final row with non-numeric data
      hit_df = hit_df[16][:-1]
      hit_df[('temp', 'Date')] = date_str
      hit_df.columns = hit_df.columns.droplevel(0)
      if len(hit_df['#']) > 1:
        hit = hit.append(hit_df)
      # scrape pitching data
      pit_df = pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=8&season={date.year}&month=1000&season1={date.year}&ind=0&team=0%2Cts&rost=0&age=0&filter=&players=0&startdate={date_str}&enddate={date_str}')
      # getting rid of the final row with non-numeric data
      pit_df = pit_df[16][:-1]
      pit_df[('temp', 'Date')] = date_str
      pit_df.columns = pit_df.columns.droplevel(0)
      if len(pit_df['#']) > 1:
        pit = pit.append(pit_df)
      date += datetime.timedelta(days = 1)
  return hit, pit

In [ ]:
#CODE USED FOR INITIAL SCRAPING

# pit.drop(columns = ['G'], inplace = True)
# # Joining hitting and pitching dataframes on team and date
# all_stats = pd.merge(hit, pit, left_on = ['Team', 'Date'], right_on = ['Team', 'Date'], how = 'inner')
# # Excludes data from days where team played a double header
# all_stats = all_stats[all_stats.GS == '1']
# all_stats.to_csv('daily_game_stats.csv') 
# files.download('daily_game_stats.csv')

In [21]:
def update_game_data():
    # Need to load in CSV identify the most recent date, scrape from most recent date to today, append
    all_stats = pd.read_csv('https://github.com/timseymour42/MLB-Build-a-Team/blob/a3774339cb04887ba2026cab07c2923b27422b60/daily_stats%20(2).csv?raw=true', header = 0, index_col = 0)
    hit, pit = collect_new_team_data(all_stats)
    if len(pit) > 0:
      pit.drop(columns = ['G'], inplace = True)
      # Joining hitting and pitching dataframes on team and date
      new_stats = pd.merge(hit, pit, left_on = ['Team', 'Date'], right_on = ['Team', 'Date'], how = 'inner')
      all_stats = pd.concat([all_stats, new_stats])
      all_stats.drop_duplicates(inplace = True)
      all_stats.to_csv('https://github.com/timseymour42/MLB-Build-a-Team/blob/a3774339cb04887ba2026cab07c2923b27422b60/daily_stats%20(2).csv')

In [12]:
update_game_data()

here
here
here
Index(['#_x', 'Team', 'G', 'PA', 'HR', 'R', 'RBI', 'SB', 'BB%', 'K%', 'ISO',
       'BABIP_x', 'AVG', 'OBP', 'SLG', 'wOBA', 'xwOBA', 'wRC+', 'BsR', 'Off',
       'Def', 'WAR_x', 'Date', '#_y', 'W', 'L', 'SV', 'GS', 'IP', 'K/9',
       'BB/9', 'HR/9', 'BABIP_y', 'LOB%', 'GB%', 'HR/FB', 'vFA (pi)', 'ERA',
       'xERA', 'FIP', 'xFIP', 'WAR_y'],
      dtype='object')


In [1]:
pip install mysqlclient

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 2.4 MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.1.1-cp37-cp37m-linux_x86_64.whl size=99980 sha256=5ee8fcfad27266cdc43c8d66f3a9cdd1295a72863597ac872a9d8d2a38d7d4c2
  Stored in directory: /root/.cache/pip/wheels/95/2d/67/2cb3f82e435fc8e055cb2761a15a0812bf086068f6fb835462
Successfully built mysqlclient


In [14]:
import MySQLdb
db=MySQLdb.connect("localhost", 'root', 'P@ssw0rd', 'mlb_db')
tblchk = db.cursor()
tblchk.execute('Drop table if exists game_data')
sql_query = '''create table game_data(gm_id int auto_increment primary key, #_x varchar(255), Team varchar(255), G varchar(255), PA varchar(255), HR varchar(255), R varchar(255), RBI varchar(255), SB varchar(255),
 BB% varchar(255), K% varchar(255), ISO varchar(255), BABIP_x varchar(255), AVG varchar(255), OBP varchar(255), SLG varchar(255), wOBA varchar(255), xwOBA varchar(255), wRC+ varchar(255), 
 BsR varchar(255), Off varchar(255), Def varchar(255), WAR_x varchar(255), Date varchar(255), #_y varchar(255), W varchar(255), L varchar(255), SV varchar(255), GS varchar(255), 
 IP varchar(255), K/9 varchar(255), BB/9 varchar(255), HR/9 varchar(255), BABIP_y varchar(255), LOB% varchar(255), GB% varchar(255), HR/FB varchar(255), vFA (pi) varchar(255), 
 ERA varchar(255), xERA varchar(255), FIP varchar(255), xFIP varchar(255), WAR_y varchar(255)'''
tblchk.execute(sql_query)

OperationalError: ignored

In [6]:
bruh = ''
for col in yo.columns:
  bruh += (col + ' varchar(255), ')

In [7]:
bruh

'#_x varchar(255), Team varchar(255), G varchar(255), PA varchar(255), HR varchar(255), R varchar(255), RBI varchar(255), SB varchar(255), BB% varchar(255), K% varchar(255), ISO varchar(255), BABIP_x varchar(255), AVG varchar(255), OBP varchar(255), SLG varchar(255), wOBA varchar(255), xwOBA varchar(255), wRC+ varchar(255), BsR varchar(255), Off varchar(255), Def varchar(255), WAR_x varchar(255), Date varchar(255), #_y varchar(255), W varchar(255), L varchar(255), SV varchar(255), GS varchar(255), IP varchar(255), K/9 varchar(255), BB/9 varchar(255), HR/9 varchar(255), BABIP_y varchar(255), LOB% varchar(255), GB% varchar(255), HR/FB varchar(255), vFA (pi) varchar(255), ERA varchar(255), xERA varchar(255), FIP varchar(255), xFIP varchar(255), WAR_y varchar(255), '

In [24]:
all_stats.to_csv('https://github.com/timseymour42/MLB-Build-a-Team/blob/a3774339cb04887ba2026cab07c2923b27422b60/daily_stats%20(2).csv')

In [12]:
yo = pd.read_csv('https://github.com/timseymour42/MLB-Build-a-Team/blob/a3774339cb04887ba2026cab07c2923b27422b60/daily_stats%20(2).csv?raw=true', header = 0, index_col = 0)

In [13]:
# Need to load in CSV identify the most recent date, scrape from most recent date to today, append
all_stats = pd.read_csv('https://github.com/timseymour42/MLB-Build-a-Team/blob/a3774339cb04887ba2026cab07c2923b27422b60/daily_stats%20(2).csv?raw=true', header = 0, index_col = 0)
hit, pit = collect_new_team_data(all_stats)
if len(pit) > 0:
  pit.drop(columns = ['G'], inplace = True)
  # Joining hitting and pitching dataframes on team and date
  new_stats = pd.merge(hit, pit, left_on = ['Team', 'Date'], right_on = ['Team', 'Date'], how = 'inner')
  print(new_stats.columns)
  all_stats = pd.concat([all_stats, new_stats])

here
here
here
Index(['#_x', 'Team', 'G', 'PA', 'HR', 'R', 'RBI', 'SB', 'BB%', 'K%', 'ISO',
       'BABIP_x', 'AVG', 'OBP', 'SLG', 'wOBA', 'xwOBA', 'wRC+', 'BsR', 'Off',
       'Def', 'WAR_x', 'Date', '#_y', 'W', 'L', 'SV', 'GS', 'IP', 'K/9',
       'BB/9', 'HR/9', 'BABIP_y', 'LOB%', 'GB%', 'HR/FB', 'vFA (pi)', 'ERA',
       'xERA', 'FIP', 'xFIP', 'WAR_y'],
      dtype='object')


In [20]:
pd.concat([all_stats, new_stats])

,#_x,Team,G,PA,HR,R,RBI,SB,BB%,K%,ISO,BABIP_x,AVG,OBP,SLG,wOBA,xwOBA,wRC+,BsR,Off,Def,WAR_x,Date,#_y,W,L,SV,GS,IP,K/9,BB/9,HR/9,BABIP_y,LOB%,GB%,HR/FB,vFA (pi),ERA,xERA,FIP,xFIP,WAR_y
0,1,STL,14,40,0,3,3,4,10.0%,27.5%,0.083,0.4,0.278,0.35,0.361,0.308,NaN,97,0.4,0.1,0.5,0.2,2015-04-05,1,1,0,1,1,9.0,12.0,2.0,0.0,0.25,100.0%,50.0%,0.0%,92.6,0.0,NaN,1.13,1.79,0.4
1,2,CHC,16,34,0,0,0,1,5.9%,35.3%,0.094,0.25,0.156,0.206,0.25,0.203,NaN,27,0.3,-2.9,0.0,-0.2,2015-04-05,2,0,1,0,1,9.0,11.0,4.0,0.0,0.4,78.6%,40.0%,0.0%,93.2,3.0,NaN,2.02,3.33,0.2
2,1,KCR,13,42,2,10,9,2,11.9%,7.1%,0.265,0.379,0.382,0.488,0.647,0.478,NaN,222,0.3,6.1,0.1,0.8,2015-04-06,20,1,0,0,1,9.0,3.0,2.0,1.0,0.167,100.0%,56.0%,14.3%,95.3,1.0,NaN,4.91,4.62,0.0
3,2,BOS,13,42,5,8,8,1,16.7%,21.4%,0.429,0.19,0.257,0.381,0.686,0.444,NaN,191,0.1,4.2,0.1,0.6,2015-04-06,4,1,0,0,1,9.0,9.0,2.0,0.0,0.136,100.0%,52.4%,0.0%,93.6,0.0,NaN,1.8,2.62,0.4
4,3,COL,15,44,2,10,10,0,2.3%,13.6%,0.286,0.412,0.381,0.395,0.667,0.451,NaN,185,0.0,4.1,0.2,0.6,2015-04-06,5,1,0,0,1,9.0,9.0,0.0,0.0,0.333,100.0%,33.3%,0.0%,90.6,0.0,NaN,1.47,2.62,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,26,ATL,15,30,1,2,2,0,3.3%,63.3%,.103,.111,.069,.100,.172,.121,NaN,-23,0.0,-4.6,0.2,-0.4,2022-08-07,15,0,1,0,1,8.0,9.00,3.38,0.00,.385,61.5%,34.6%,0.0%,97.3,5.63,NaN,2.24,4.28,0.2
26,27,HOU,15,29,0,0,0,0,3.4%,34.5%,.000,.111,.071,.103,.071,.085,NaN,-44,-0.1,-5.2,0.2,-0.4,2022-08-07,14,0,1,0,1,8.0,6.75,2.25,1.13,.280,100.0%,23.1%,6.3%,94.5,1.13,NaN,3.99,5.33,0.2
27,28,DET,16,30,0,0,0,0,0.0%,30.0%,.067,.143,.100,.100,.167,.113,NaN,-25,-0.4,-5.0,0.0,-0.4,2022-08-07,12,0,1,0,1,9.0,9.00,6.00,0.00,.346,53.3%,23.1%,0.0%,92.8,7.00,NaN,3.12,5.59,0.2
28,29,SDP,14,30,0,0,0,0,6.7%,13.3%,.000,.083,.071,.133,.071,.105,NaN,-32,0.1,-4.8,-0.5,-0.4,2022-08-07,20,0,1,0,1,8.0,7.88,0.00,2.25,.346,85.4%,42.9%,20.0%,94.7,4.50,NaN,4.62,3.22,0.0


In [18]:
pd.set_option('display.max_columns', None)

In [19]:
pd.merge(hit, pit, left_on = ['Team', 'Date'], right_on = ['Team', 'Date'], how = 'inner')

,#_x,Team,G,PA,HR,R,RBI,SB,BB%,K%,ISO,BABIP_x,AVG,OBP,SLG,wOBA,xwOBA,wRC+,BsR,Off,Def,WAR_x,Date,#_y,W,L,SV,GS,IP,K/9,BB/9,HR/9,BABIP_y,LOB%,GB%,HR/FB,vFA (pi),ERA,xERA,FIP,xFIP,WAR_y
0,1,PHI,15,46,4,13,13,0,13.0%,13.0%,.405,.333,.351,.478,.757,.514,NaN,236,0.0,7.3,-0.8,0.8,2022-08-07,11,1,0,0,1,9.0,8.00,3.00,0.00,.280,90.0%,40.0%,0.0%,93.8,1.00,NaN,2.34,3.49,0.2
1,2,STL,14,44,2,12,12,0,18.2%,15.9%,.265,.346,.324,.455,.588,.442,NaN,192,0.0,4.7,0.4,0.7,2022-08-07,10,1,0,1,1,9.0,13.00,4.00,1.00,.517,61.2%,40.0%,9.1%,94.4,9.00,NaN,3.34,3.71,0.3
2,3,NYY,15,48,1,9,9,0,8.3%,27.1%,.167,.517,.381,.438,.548,.417,NaN,178,-0.1,4.1,0.5,0.6,2022-08-07,29,0,1,0,1,8.0,7.88,9.00,2.25,.346,46.5%,32.1%,18.2%,97.3,13.50,NaN,7.99,6.78,-0.2
3,4,KCR,14,40,1,13,12,0,15.0%,15.0%,.323,.385,.355,.450,.677,.458,NaN,206,-0.6,4.1,-0.4,0.5,2022-08-07,25,1,0,0,1,9.0,6.00,3.00,2.00,.296,78.4%,55.2%,28.6%,95.0,5.00,NaN,5.67,3.94,-0.1
4,5,ARI,15,36,2,6,5,0,16.7%,8.3%,.276,.280,.310,.417,.586,.423,NaN,176,-0.3,2.8,0.4,0.5,2022-08-07,17,1,0,1,1,9.0,5.00,6.00,0.00,.333,75.0%,26.7%,0.0%,94.8,4.00,NaN,4.00,5.65,0.1
5,6,CHW,15,45,1,8,7,0,4.4%,17.8%,.163,.412,.349,.378,.512,.385,NaN,157,0.0,2.8,-0.2,0.4,2022-08-07,18,1,0,0,1,9.0,10.00,5.00,1.00,.333,95.2%,40.0%,11.1%,93.4,2.00,NaN,4.00,4.04,0.1
6,7,LAD,14,35,2,4,4,1,0.0%,20.0%,.257,.346,.314,.314,.571,.378,NaN,151,0.3,2.2,0.2,0.4,2022-08-07,9,1,0,0,1,9.0,4.00,2.00,0.00,.083,100.0%,45.8%,0.0%,92.2,0.00,NaN,2.89,4.54,0.3
7,8,TEX,14,40,1,2,2,0,12.5%,25.0%,.114,.333,.257,.350,.371,.324,NaN,116,0.2,0.7,0.8,0.3,2022-08-07,19,0,1,0,1,9.0,8.00,2.00,1.00,.412,57.7%,42.9%,12.5%,94.1,6.00,NaN,3.45,3.32,0.1
8,9,SEA,13,36,1,6,6,0,13.9%,11.1%,.194,.269,.258,.361,.452,.356,NaN,142,-0.1,1.5,0.0,0.3,2022-08-07,2,1,0,1,1,9.0,11.00,1.00,0.00,.348,66.7%,59.1%,0.0%,91.2,3.00,NaN,1.00,1.66,0.4
9,10,SFG,16,41,3,6,6,1,12.2%,26.8%,.278,.273,.250,.341,.528,.358,NaN,128,0.2,1.4,-0.6,0.2,2022-08-07,21,1,0,1,1,9.0,6.00,4.00,1.00,.276,79.4%,63.3%,14.3%,95.0,4.00,NaN,4.89,4.60,0.0


In [14]:
new_stats

,#_x,Team,G,PA,HR,R,RBI,SB,BB%,K%,...,BABIP_y,LOB%,GB%,HR/FB,vFA (pi),ERA,xERA,FIP,xFIP,WAR_y
0,1,PHI,15,46,4,13,13,0,13.0%,13.0%,...,.280,90.0%,40.0%,0.0%,93.8,1.00,NaN,2.34,3.49,0.2
1,2,STL,14,44,2,12,12,0,18.2%,15.9%,...,.517,61.2%,40.0%,9.1%,94.4,9.00,NaN,3.34,3.71,0.3
2,3,NYY,15,48,1,9,9,0,8.3%,27.1%,...,.346,46.5%,32.1%,18.2%,97.3,13.50,NaN,7.99,6.78,-0.2
3,4,KCR,14,40,1,13,12,0,15.0%,15.0%,...,.296,78.4%,55.2%,28.6%,95.0,5.00,NaN,5.67,3.94,-0.1
4,5,ARI,15,36,2,6,5,0,16.7%,8.3%,...,.333,75.0%,26.7%,0.0%,94.8,4.00,NaN,4.00,5.65,0.1
5,6,CHW,15,45,1,8,7,0,4.4%,17.8%,...,.333,95.2%,40.0%,11.1%,93.4,2.00,NaN,4.00,4.04,0.1
6,7,LAD,14,35,2,4,4,1,0.0%,20.0%,...,.083,100.0%,45.8%,0.0%,92.2,0.00,NaN,2.89,4.54,0.3
7,8,TEX,14,40,1,2,2,0,12.5%,25.0%,...,.412,57.7%,42.9%,12.5%,94.1,6.00,NaN,3.45,3.32,0.1
8,9,SEA,13,36,1,6,6,0,13.9%,11.1%,...,.348,66.7%,59.1%,0.0%,91.2,3.00,NaN,1.00,1.66,0.4
9,10,SFG,16,41,3,6,6,1,12.2%,26.8%,...,.276,79.4%,63.3%,14.3%,95.0,4.00,NaN,4.89,4.60,0.0


# Scraping Player Data

In [ ]:
def scrape_player_data():
    # beginning of sample is 1900
    year = 1900
    wrc = pd.DataFrame()
    pitch = pd.DataFrame()
    field = pd.DataFrame()
    # sustainable way of changing year without change in code
    while year < datetime.datetime.now().year + 1:
        for num in range(int(pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=&page=1_50')[16].columns[0][0][-8:-6].strip())):
            # scrape hitting data
            if (num < 1):
                temp = pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=&page={str(num + 1)}_50')[16][:-1]   
                temp.columns = temp.columns.droplevel(0)
                wrc_df = temp
            else:
                temp = (pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=&page={str(num + 1)}_50')[16][:-1])
                temp.columns = temp.columns.droplevel(0)
                wrc_df = wrc_df.append(temp)
            # getting rid of the final row with non-numeric data above
        wrc_df['Season'] = year
        wrc = wrc.append(wrc_df)
        # scrape pitching data
        for num in range(int(pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate={year}-01-01&enddate={year}-12-31&sort=21,d&page=1_50')[16].columns[0][0][-8:-6].strip())):
            # scrape hitting data
            if (num < 1):
                temp = pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate={year}-01-01&enddate={year}-12-31&sort=21,d&page={str(num + 1)}_50')[16][:-1]   
                temp.columns = temp.columns.droplevel(0)
                pitch_df = temp
            else:
                temp = (pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate={year}-01-01&enddate={year}-12-31&sort=21,d&page={str(num + 1)}_50')[16][:-1])
                temp.columns = temp.columns.droplevel(0)
                pitch_df = pitch_df.append(temp)

            # getting rid of the final row with non-numeric data above
        pitch_df['Season'] = year
        pitch = pitch.append(pitch_df)
        year+=1
    return wrc, pitch

1900
1905
1910
1915
1920
1925
1930
1935
1940
1945
1950
1955
1960
1965
1970
1975
1980
1985
1990
1995
2000
2005
2010
2015
2020


In [ ]:
def string_to_num(string):
    if(type(string) == str):
        if('%' in string):
            string = string.replace('%', '')
    return float(string)

In [ ]:
def clean_player_data(hit_df, pitch_df):
  '''
  function intended to make statistics numerical, manually calculate statistics, and set the indices to Name and Season

  Args:
    wrc (pd.DataFrame) contains individual player data by season
    pitch (pd.DataFrame) contains individual pitcher data by season

  Returns wrc, pitch as clean datasets for use in App'''

  # applying the function to each column to ensure all data points are numerical
  for col in hit_df.columns:
      if col not in ['Name', 'Team', 'Season', 'GB', 'Pos']:
          hit_df[col] = hit_df[col].apply(string_to_num)
  for col in pitch_df.columns:
      if col not in ['Name', 'Team', 'Season', 'GB']:
          pitch_df[col] = pitch_df[col].apply(string_to_num)
  #Determining home runs allowed for each player for easier calculation
  pitch_df['HR'] = pitch_df['HR/9'] * pitch_df['IP'] * 9
  #Determining total bases for each player for more accurate slugging percentage calculation
  # First must find at bats by subtracting walks using walk percentage
  # Calculation ignores HBP
  hit_df['AB'] = hit_df['PA'] * (1 - (hit_df['BB%'] * .01))
  # Calculation necessary for determining slugging percentage over multiple seasons
  hit_df['TB'] = hit_df['SLG'] * hit_df['AB']
  pitch_df.set_index(['Name', 'Season'], inplace = True)
  hit_df.set_index(['Name', 'Season'], inplace = True)
  return hit_df, pitch_df

In [ ]:
def add_new_player_data(hit_df, pit_df):
  # Setting up current CSV data to be appended to
  year = hit_df['Season'].max()
  # Excluding current year for freshly scraped aggregates
  hit_df = hit_df[hit_df.Season != year]
  pit_df = pit_df[pit_df.Season != year]
  wrc = pd.DataFrame()
  pitch = pd.DataFrame()

  while year < datetime.datetime.now().year + 1:
    for num in range(int(pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=&page=1_50')[16].columns[0][0][-8:-6].strip())):
        # scrape hitting data
        if (num < 1):
            temp = pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=&page={str(num + 1)}_50')[16][:-1]   
            temp.columns = temp.columns.droplevel(0)
            wrc_df = temp
        else:
            temp = (pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=&page={str(num + 1)}_50')[16][:-1])
            temp.columns = temp.columns.droplevel(0)
            wrc_df = wrc_df.append(temp)
        # getting rid of the final row with non-numeric data above
    wrc_df['Season'] = year
    wrc = wrc.append(wrc_df)
    # scrape pitching data
    for num in range(int(pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate={year}-01-01&enddate={year}-12-31&sort=21,d&page=1_50')[16].columns[0][0][-8:-6].strip())):
        # scrape hitting data
        if (num < 1):
            temp = pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate={year}-01-01&enddate={year}-12-31&sort=21,d&page={str(num + 1)}_50')[16][:-1]   
            temp.columns = temp.columns.droplevel(0)
            pitch_df = temp
        else:
            temp = (pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate={year}-01-01&enddate={year}-12-31&sort=21,d&page={str(num + 1)}_50')[16][:-1])
            temp.columns = temp.columns.droplevel(0)
            pitch_df = pitch_df.append(temp)
        # getting rid of the final row with non-numeric data above
    pitch_df['Season'] = year
    pitch = pitch.append(pitch_df)
    year+=1
  hit_a, pitch_a = clean_player_data(wrc, pitch)
  hit_df = pd.concat([hit_df, hit_a.reset_index()])
  pit_df = pd.concat([pit_df, pitch_a.reset_index()])
  return hit_df, pit_df


In [ ]:
# CODE FROM INITIAL SCRAPING

# hit_df.to_csv('hitters_yearly.csv') 
# pitch_df.to_csv('pitchers_yearly.csv')
# files.download('hitters_yearly.csv')
# files.download('pitchers_yearly.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def update_players_data():
    # Load in each CSV, clean
    # Drop all records from current year, scrape all records from current year, append them
    # try drop_duplicates
    # Save each CSV to GitHub
    hit_df = pd.read_csv('https://github.com/timseymour42/MLB-Build-a-Team/blob/5548a60b92575ee19b159c791934630cbd9f72d3/hitters_yearly.csv?raw=true', header = 0)
    pitch_df = pd.read_csv('https://github.com/timseymour42/MLB-Build-a-Team/blob/5548a60b92575ee19b159c791934630cbd9f72d3/pitchers_yearly.csv?raw=true', header = 0)
    hit, pit = add_new_player_data(hit_df, pitch_df)
    hit_df.drop_duplicates(inplace = True)
    pitch_df.drop_duplicates(inplace = True)
    hit_df.to_csv('https://github.com/timseymour42/MLB-Build-a-Team/blob/5548a60b92575ee19b159c791934630cbd9f72d3/hitters_yearly.csv')
    pitch_df.to_csv('https://github.com/timseymour42/MLB-Build-a-Team/blob/5548a60b92575ee19b159c791934630cbd9f72d3/pitchers_yearly.csv')

,Name,Season,#,Team,G,PA,HR,R,RBI,SB,...,SLG,wOBA,xwOBA,wRC+,BsR,Off,Def,WAR,AB,TB
0,Honus Wagner,1900,1.0,PIT,135.0,580.0,4.0,107.0,100.0,38.0,...,0.573,0.465,NaN,166.0,2.6,57.4,-3.5,6.9,538.820,308.743860
1,Elmer Flick,1900,2.0,PHI,138.0,623.0,11.0,106.0,110.0,35.0,...,0.545,0.459,NaN,163.0,1.1,57.0,-10.8,6.3,566.930,308.976850
2,Jesse Burkett,1900,3.0,STL,141.0,643.0,7.0,88.0,68.0,32.0,...,0.474,0.429,NaN,144.0,-0.1,40.5,-3.9,5.6,581.272,275.522928
3,John McGraw,1900,4.0,STL,99.0,447.0,2.0,84.0,33.0,29.0,...,0.416,0.458,NaN,161.0,0.1,38.9,4.1,5.4,362.070,150.621120
4,Kip Selbach,1900,5.0,NYG,141.0,611.0,4.0,98.0,68.0,36.0,...,0.461,0.422,NaN,143.0,1.4,39.3,-5.0,5.3,538.902,248.433822
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353,Nick Castellanos,2022,1354.0,PHI,106.0,441.0,10.0,45.0,54.0,4.0,...,0.385,0.301,0.306,91.0,-3.9,-8.8,-17.4,-1.2,417.186,160.616610
1354,Robinson Chirinos,2022,1355.0,BAL,51.0,169.0,3.0,8.0,17.0,1.0,...,0.282,0.245,0.242,54.0,-0.6,-9.4,-7.7,-1.2,154.973,43.702386
1355,Yoshi Tsutsugo,2022,1356.0,PIT,50.0,193.0,2.0,11.0,19.0,0.0,...,0.229,0.221,0.255,38.0,-0.7,-14.6,-4.3,-1.3,174.086,39.865694
1356,Jonathan Villar,2022,1357.0,- - -,59.0,220.0,3.0,25.0,18.0,7.0,...,0.302,0.250,0.247,56.0,2.1,-9.0,-11.3,-1.3,205.040,61.922080


# Collecting team data to compare model predictions to actual full season win totals

- key question is what model or combination of models minimizes error in predicting team success historically

In [ ]:
def collect_team_data_yearly(year):

  '''
  Args:
    year (integer): year to start collecting data from
  Collecting team data to use as testing data
  '''
  
  wrc = pd.DataFrame()
  pitch = pd.DataFrame()
  field = pd.DataFrame()
  # sustainable way of changing year without change in code
  while year < datetime.datetime.now().year + 1:
      # scrape hitting data
      wrc_df = pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0,ts&rost=0&age=0&filter=&players=0&startdate=&enddate=')
      # getting rid of the final row with non-numeric data
      wrc_df = wrc_df[16][:-1]
      wrc_df[('temp', 'Season')] = year
      wrc_df.columns = wrc_df.columns.droplevel(0)
      wrc = pd.concat([wrc, wrc_df], axis = 0)
      # scrape pitching data
      pitch_df = pd.read_html(f'https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0,ts&rost=0&age=0&filter=&players=0&startdate=&enddate=')
      # getting rid of the final row with non-numeric data
      pitch_df = pitch_df[16][:-1]
      pitch_df[('temp', 'Season')] = year
      pitch_df.columns = pitch_df.columns.droplevel(0)
      pitch = pd.concat([pitch, pitch_df], axis = 0)
      year += 1
  return wrc, pitch

w, p = collect_team_data_yearly(1900)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# CODE USED FOR INITIAL SCRAPING

# team_data.to_csv('team_yearly_data.csv')
# files.download('team_yearly_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Load in each CSV
# Drop all records from current year, scrape all records from current year, append them
# try drop_duplicates
# Save each CSV to GitHub
def update_team_data():
    # Load in each CSV, clean
    # Drop all records from current year, scrape all records from current year, append them
    # try drop_duplicates
    # Save each CSV to GitHub
    team_data = pd.read_csv('https://github.com/timseymour42/MLB-Build-a-Team/blob/8d552de89f4daf8a9aa27edde95179f3bb192258/team_yearly_data.csv?raw=true', header = 0)
    curr_year = team_data['Season'].max()
    w, p = collect_team_data_yearly(curr_year)
    new_team_data = pd.merge(w, p, left_on = ['Season', 'Team'], right_on = ['Season', 'Team'], how = 'outer')
    # applying the function to each column to ensure all data points are numerical
    for col in new_team_data.columns:
        if col not in ['Team', 'Season', 'GB']:
          new_team_data[col] = new_team_data[col].apply(string_to_num)
    new_team_data['W'] = new_team_data['W'] * (162 / team_data['GS'])
    new_team_data = pd.concat([team_data, new_team_data])
    new_team_data.drop_duplicates(inplace = True)
    new_team_data.to_csv('https://github.com/timseymour42/MLB-Build-a-Team/blob/8d552de89f4daf8a9aa27edde95179f3bb192258/team_yearly_data.csv')